# Myria Python & Jupyter

<img src="overview.png" style="height: 300px"/>

### To install `Myria-Python`:

```
pip install myria-python
```

### Or:

```
git clone https://github.com/uwescience/myria-python
cd myria-python
sudo python setup.py install
```



## 1. Connecting to Myria

In [1]:
from myria import *
import numpy

# Load Myria extensions
%load_ext myria

# Create Jupyter Connection
%connect http://localhost:8753

/home/gridsan/dhutchison/.local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/gridsan/dhutchison/.local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
# Alternatively, create an ordinary Python connection to the Myria demo cluster
connection = MyriaConnection(rest_url='http://node-109:8753')
# Use this as the default connection
MyriaRelation.DefaultConnection = connection

## 2. Myria: Connections, Relations, and Queries (and Schemas and Plans)

In [ ]:
connection.datasets()

In [4]:
# How many datasets are there on the server?
print len(connection.datasets())

422


In [5]:
# Let's look at the first dataset...
dataset = connection.datasets()[0]
print dataset['relationKey']['relationName']
print dataset['created']

AboveNotBelow
2016-08-13T04:39:02.106-04:00


In [ ]:
import inspect
inspect.getsourcelines(MyriaRelation) 

In [11]:
# View data stored in this relation
relation = MyriaRelation(dataset['relationKey'])
relation

,Al_D_CONC_BOTTLE,Ba_D_CONC_BOTTLE,CTDOXY,Cd_D_CONC_BOTTLE,DIC,Fe_D_CONC_BOTTLE,Fe_D_CONC_BOTTLE_FIA,Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV,H2O_18_D_DELTA_BOTTLE,H2O_2_D_DELTA_BOTTLE,La_D_CONC_BOTTLE,Library,Mn_D_CONC_BOTTLE,Mn_D_CONC_BOTTLE_STANDARD_DEV,Mo_D_CONC_BOTTLE,NITRAT,NITRIT,Ni_D_CONC_BOTTLE,OXYGEN,PHSPHT,Pa_231_D_CONC_BOTTLE,Pa_231_D_CONC_BOTTLE_STANDARD_DEV,Pb_D_CONC_BOTTLE,SILCAT,STANDARD_DEV,Station,TALK,Th_230_D_CONC_BOTTLE,Th_230_D_CONC_BOTTLE_STANDARD_DEV,Th_232_D_CONC_BOTTLE,Th_232_D_CONC_BOTTLE_STANDARD_DEV,Th_234_D_CONC_BOTTLE,Th_234_D_CONC_BOTTLE_STANDARD_DEV,U_D_CONC_BOTTLE,Y_D_CONC_BOTTLE,Zn_D_CONC_BOTTLE,depth,pressure,sal,temp
0,0.000000,0.0000,217.4,0.00000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000,S0072,0.000000,0.00000,0.000,0.0000,0.000000,0.000,0.0,0.000,0.0,0.0,0.000,0.000000,0.000000,20,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,0.00000,0.000,50.0,40.0,36.667250,19.676625
1,1.985365,46.9873,267.7,0.11765,2088.8,0.09,0.080975,0.008295,0.12,0.94,17.265,S0214,0.668295,0.00683,126.356,6.8075,0.100000,3.685,241.9,0.530,0.0,0.0,21.615,3.175000,0.019515,3,2305.2,0.0,0.0,0.0,0.0,21.4829,0.53658,12.4400,119.74000,0.515,10.0,10.0,34.813499,11.465000
2,0.000000,0.0000,228.0,0.00000,2045.8,0.00,0.000000,0.000000,0.00,0.00,0.000,S0237,0.000000,0.00000,0.000,0.0000,0.000000,0.000,230.2,0.070,0.0,0.0,0.000,0.800000,0.000000,7,2334.9,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,0.00000,0.000,9.0,2.0,35.369999,20.258000
3,20.400000,41.2976,197.2,0.00170,2002.6,0.34,0.263420,0.019510,0.79,6.76,19.090,S0250,1.629270,0.01854,134.217,0.0000,0.020000,2.140,197.2,0.035,0.0,0.0,20.580,0.365000,0.078050,18,2337.6,0.0,0.0,0.0,0.0,34.7610,0.74146,12.9024,105.47000,3.490,9.0,2.0,35.698501,28.567500
4,4.419510,48.7639,287.8,0.05280,2073.1,0.52,0.497560,0.019510,-0.26,-1.78,11.050,S0209,0.087800,0.00098,118.365,13.1450,0.165000,4.880,285.6,1.010,0.0,0.0,14.940,0.000000,0.117070,1,2277.8,0.0,0.0,0.0,0.0,25.0829,0.60488,11.6117,114.88000,0.210,10.0,10.0,33.977501,10.093000
5,12.614600,42.7746,193.9,0.00130,2078.3,0.10,0.078050,0.000000,0.98,7.41,16.500,S0191,1.658540,0.03707,148.461,0.0000,0.020000,2.470,0.0,0.095,0.0,0.0,16.270,0.445000,0.107320,14,2429.7,0.0,0.0,0.0,0.0,36.6244,0.75122,12.9766,101.04000,0.170,10.0,10.0,37.017001,28.117500
6,3.292685,38.1073,255.2,0.01635,2068.5,0.14,0.138050,0.005370,0.18,4.98,20.865,S0001,0.936585,0.00878,144.160,0.3000,0.053333,2.815,210.3,0.090,0.0,0.0,20.170,0.753333,0.043900,10,2310.8,0.0,0.0,0.0,0.0,35.4927,0.80000,12.6078,122.78000,0.165,10.0,10.0,35.038667,15.072333
7,42.819500,0.0000,194.8,0.00050,2016.6,1.44,1.532680,0.049760,0.00,0.00,18.250,S0152,3.531710,0.02634,0.000,0.0000,0.015000,2.000,0.0,0.010,0.0,0.0,22.010,1.100000,0.136580,27,2367.2,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,138.85001,0.020,12.0,10.0,36.141500,28.409000
8,3.292685,38.1073,255.2,0.01635,2068.5,0.14,0.138050,0.005370,0.18,4.98,20.865,S0243,0.936585,0.00878,144.160,0.3000,0.053333,2.815,210.3,0.090,0.0,0.0,20.170,0.753333,0.043900,10,2310.8,0.0,0.0,0.0,0.0,35.4927,0.80000,12.6078,122.78000,0.165,10.0,2.0,35.038667,15.072333
9,41.424400,0.0000,203.5,0.00050,0.0,1.77,1.430240,0.020490,0.00,0.00,17.600,S0147,2.760980,0.03805,0.000,0.0000,0.020000,2.010,197.4,0.010,0.0,0.0,23.610,1.000000,0.136580,25,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,138.10001,0.060,10.0,10.0,36.463001,27.220000


In [14]:
relation.to_dict()[0]['CTDOXY']

287.8

### Uploading data

In [ ]:
%%query

-- Load from S3
florida = load("https://s3-us-west-2.amazonaws.com/myria-demo-data/fl_insurance_sample_2.csv",
csv(schema(
            id:int,
            geo:string,
            granularity:int,
            deductable:float,
            policyID:int, 
            construction:string,
            line:string,
            county:string,
            state:string,
            longitude:float,
            latitude:float,
            fl_site_deductible:float,
            hu_site_deductible:float,
            eq_site_deductible:float,
            tiv_2012:float,
            tiv_2011:float,
            fr_site_limit:float,
            fl_site_limit:float,
            hu_site_limit:float,
            eq_site_limit:float), skip=1));


clay_county = [from florida where county = 'CLAY COUNTY' emit *];

store(clay_county, insurance);

In [ ]:
# Alternatively, you can upload directly from a Python string
name = {'userName': 'Brandon', 'programName': 'Demo', 'relationName': 'Books'}
schema = { "columnNames" : ["name", "pages"],
           "columnTypes" : ["STRING_TYPE","LONG_TYPE"] }

data = """Brave New World,288
Nineteen Eighty-Four,376
We,256"""

result = connection.upload_file(
    name, schema, data, delimiter=',', overwrite=True)

MyriaRelation(result['relationKey'], connection=connection)

In [ ]:
#Or, load using the myria_upload command-line utility
!wget https://s3-us-west-2.amazonaws.com/myria-demo-data/books.csv
!myria_upload --hostname demo.myria.cs.washington.edu --port 8753 --no-ssl --user Brandon --program Demo --relation Demo --overwrite books.csv

### Working with relations

In [ ]:
# Using the previously-stored insurance relation
MyriaRelation("insurance")

In [ ]:
# View details about this relation
relation = MyriaRelation("insurance")
print len(relation)
print relation.created_date
print relation.schema.names

### Working Locally with Relations

In [ ]:
# 1: Download as a Python dictionary
d = MyriaRelation("insurance").to_dict()
print 'First entry returned: %s' % d[0]['county']

In [ ]:
# 2: Download as a Pandas DataFrame
df = MyriaRelation("insurance").to_dataframe()
print '%d entries with nonzero deductable' % len(df[df.eq_site_deductible > 0]) 

In [ ]:
# 3: Download as a DataFrame and convert to a numpy array
array = MyriaRelation("insurance").to_dataframe().as_matrix()
print 'Mean site limit = %d' % array[:,4].mean()

## Working with queries

In [ ]:
%%query --Embed MyriaL in Jupyter notebook by using the "%%query" prefix 

insurance = scan(insurance);

descriptives = [from insurance emit min(eq_site_deductible) as min_deductible, 
                                    max(eq_site_deductible) as max_deductible, 
                                    avg(eq_site_deductible) as mean_deductible, 
                                    stdev(eq_site_deductible) as stdev_deductible];

store(descriptives, descriptives);

In [ ]:
# Grab the results of the most recent execution
query = _
or_this_works_too = _45

In [ ]:
query

### Single-line queries may be treated like Python expressions

In [ ]:
query = %datalog Just500(column0, 500) :- TwitterK(column0, 500)%
print query.status
query

## 5. Variable Binding

In [ ]:
low, high, destination = 543, 550, 'BoundRelation'

The tokens `@low`, `@high`, and `@destination` are bound to their values:

In [ ]:
%%query
T1 = scan(TwitterK);
T2 = [from T1 where $0 > @low and $0 < @high emit $1 as x];
store(T2, @destination);

# Deploying Myria in an Amazon Cluster!

## 1. Installing the Myria CLI

```
# From the command line, execute:
sudo pip install myria-cluster
```

## 2. Launching Clusters

In [ ]:
!myria-cluster create my-cluster

## 3. Connecting to the Cluster via Python

You can connect to the new cluster by using the MyriaX REST endpoint URL.  In the example above, this is listed as http://ec2-50-112-33-121.us-west-2.compute.amazonaws.com:8753.

In [ ]:
# Substitute your MyriaX REST URL here!
%connect http://ec2-52-1-38-182.compute-1.amazonaws.com:8753 

# Where to find more information:

#### Documentation
[Myria Website](http://myria.cs.washington.edu/)<br /> 
[Myria Python](http://myria.cs.washington.edu/docs/myria-python/)<br /> 
[Additional Language Documentation](http://myria.cs.washington.edu/docs/myrial.html)<br /> 
[This Notebook](https://github.com/uwescience/myria-python/blob/master/ipnb%20examples/myria.ipynb) 

#### Repositories
[Myria](https://github.com/uwescience/myria)<br /> 
[Myria-Python](https://github.com/uwescience/myria-python)<br /> 
[Myria-Cluster](https://github.com/uwescience/myria-ec2-ansible)

#### Mailing List
[myria-users@cs.washington.edu](mailto:myria-users@cs.washington.edu)

## Jupyter
[Homepage](http://jupyter.org/)

## Pandas/Dataframes
[Homepage](http://pandas.pydata.org/)